In [ ]:
import boto3
from botocore.client import Config
import os

In [ ]:
# MinIO connection details
minio_endpoint = os.getenv('AWS_S3_ENDPOINT')
access_key = os.getenv('AWS_ACCESS_KEY_ID')
secret_key = os.getenv('AWS_SECRET_ACCESS_KEY')
bucket_name = os.getenv('AWS_S3_BUCKET')
region = os.getenv('AWS_DEFAULT_REGION')

# Initialize the S3 client
s3_client = boto3.client(
    's3',
    endpoint_url=minio_endpoint,
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    config=Config(signature_version='s3v4')
)

# List all buckets
buckets = s3_client.list_buckets()
print("Buckets:")
for bucket in buckets['Buckets']:
    print(f"  {bucket['Name']}")

# List objects in a specific bucket
objects = s3_client.list_objects_v2(Bucket=bucket_name)
print(f"\nObjects in bucket '{bucket_name}':")
if 'Contents' in objects:
    for obj in objects['Contents']:
        print(f"  {obj['Key']}")
else:
    print("  No objects found.")

In [ ]:
# Get working directory

# Download items in bucket
local_download_path = os.getcwd()
if 'Contents' in objects:
    for obj in objects['Contents']:
        object_key = obj['Key']
        local_file_path = os.path.join(local_download_path, object_key)

        # Ensure local directory exists
        os.makedirs(os.path.dirname(local_file_path), exist_ok=True)

        # Download the file
        s3_client.download_file(bucket_name, object_key, local_file_path)
        print(f"Downloaded {object_key} to {local_file_path}")
else:
    print("No objects found in the bucket.")